# FastAPI upload server (payload_video.ipynb)

Notebook ini menyediakan server FastAPI yang menerima upload video (multipart) di `/upload_file` dan menerima JSON payload di `/upload`.

Langkah eksekusi:
1. Jalankan cell instalasi dependensi
2. Jalankan cell setup direktori
3. Jalankan cell definisi server
4. Jalankan cell start server (ngrok akan dicoba jika tersedia)

Hasil: file yang diupload akan disimpan di folder `uploads/` dan payload JSON yang dikirim ke `/upload` akan disimpan di `received_payloads/`.

In [1]:
# Install dependencies (jalankan sekali)
!pip install --quiet fastapi uvicorn nest-asyncio pyngrok python-multipart


[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
# Siapkan direktori untuk upload dan payload
import os
ROOT_DIR = os.getcwd()
UPLOAD_DIR = os.path.join(ROOT_DIR, 'uploads')
PAYLOAD_DIR = os.path.join(ROOT_DIR, 'received_payloads')
os.makedirs(UPLOAD_DIR, exist_ok=True)
os.makedirs(PAYLOAD_DIR, exist_ok=True)
print('Upload dir:', UPLOAD_DIR)
print('Payload dir:', PAYLOAD_DIR)

Upload dir: d:\Coding\Interview_Assesment_System-main\uploads
Payload dir: d:\Coding\Interview_Assesment_System-main\received_payloads


In [3]:
# Definisikan FastAPI app dengan endpoint /upload_file, /upload, /delete_file dan form tester /upload_form
from fastapi import FastAPI, UploadFile, File, Request, HTTPException
from fastapi.middleware.cors import CORSMiddleware
from fastapi.responses import JSONResponse, HTMLResponse
from fastapi.staticfiles import StaticFiles
import uuid, shutil, json, os
from datetime import datetime

app = FastAPI(title='AI Interview Upload Server (notebook)')

app.add_middleware(
    CORSMiddleware,
    allow_origins=['*'],
    allow_credentials=True,
    allow_methods=['*'],
    allow_headers=['*'],
)

# Mount static folder 'uploads' sehingga file bisa diakses via /uploads/<name>
app.mount('/uploads', StaticFiles(directory=UPLOAD_DIR), name='uploads')
# Mount folder payload agar payload JSON dapat diakses via /payloads/<name>
app.mount('/payloads', StaticFiles(directory=PAYLOAD_DIR), name='payloads')

# global state: simpan info payload terakhir yang diterima
last_payload_info = None  # akan di-set ketika /upload dipanggil

@app.post('/upload_file')
async def upload_file(request: Request, file: UploadFile = File(...)):
    try:
        # simpan file dengan nama unik
        ext = os.path.splitext(file.filename)[1] or ''
        safe_name = f"{datetime.utcnow().strftime('%Y%m%d%H%M%S')}_{uuid.uuid4().hex}{ext}"
        dest_path = os.path.join(UPLOAD_DIR, safe_name)
        with open(dest_path, 'wb') as buffer:
            shutil.copyfileobj(file.file, buffer)
        base_url = str(request.base_url).rstrip('/')
        file_url = f"{base_url}/uploads/{safe_name}"
        # kembalikan url dan safe filename agar client bisa menghapus jika perlu
        return JSONResponse({'success': True, 'url': file_url, 'name': safe_name})
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

@app.delete('/delete_file')
async def delete_file(payload: dict):
    try:
        # payload: { 'name': '<safe_name>' }
        name = payload.get('name') if isinstance(payload, dict) else None
        if not name:
            raise HTTPException(status_code=400, detail='Missing file name')
        file_path = os.path.join(UPLOAD_DIR, name)
        if os.path.exists(file_path):
            os.remove(file_path)
            return JSONResponse({'success': True, 'deleted': name})
        else:
            return JSONResponse({'success': False, 'error': 'file not found'}, status_code=404)
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

@app.post('/upload')
async def receive_payload(request: Request, payload: dict):
    try:
        # simpan payload ke file unik
        fname = f"payload_{datetime.utcnow().strftime('%Y%m%d%H%M%S')}_{uuid.uuid4().hex}.json"
        dest = os.path.join(PAYLOAD_DIR, fname)
        with open(dest, 'w', encoding='utf-8') as f:
            json.dump(payload, f, ensure_ascii=False, indent=2)
        base_url = str(request.base_url).rstrip('/')
        payload_url = f"{base_url}/payloads/{fname}"
        # update global state agar bisa diakses via endpoint lain
        global last_payload_info
        last_payload_info = {
            'name': fname,
            'url': payload_url,
            'saved_at': datetime.utcnow().isoformat()
        }
        # kembalikan info termasuk url dan redirect ke halaman dashboard (URL absolut)
        dashboard_url = f"halaman_dasboard.html"
        return JSONResponse({'success': True, 'saved_as': fname, 'url': payload_url, 'redirect': dashboard_url})
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

# Endpoint untuk mengambil info payload terakhir yang diterima
@app.get('/last_payload')
async def get_last_payload():
    if last_payload_info is None:
        return JSONResponse({'success': False, 'message': 'No payload received yet'}, status_code=404)
    return JSONResponse({'success': True, 'last_payload': last_payload_info})

# Endpoint untuk membaca isi payload terakhir (jika ada)
@app.get('/last_payload/content')
async def get_last_payload_content():
    if last_payload_info is None:
        return JSONResponse({'success': False, 'message': 'No payload received yet'}, status_code=404)
    fp = os.path.join(PAYLOAD_DIR, last_payload_info['name'])
    if not os.path.exists(fp):
        return JSONResponse({'success': False, 'message': 'File not found'}, status_code=404)
    try:
        with open(fp, 'r', encoding='utf-8') as f:
            data = json.load(f)
        return JSONResponse({'success': True, 'content': data})
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

@app.get('/')
async def index():
    return {'message': 'AI Interview Upload Server (notebook) running. Use /upload_file and /upload endpoints.'}

# simple HTML form for manual testing (GET).
@app.get('/upload_form')
async def upload_form():
    html = '''
    <html>
      <head><meta charset="utf-8"><title>Upload Form</title></head>
      <body>
        <h3>Test upload to /upload_file (multipart POST)</h3>
        <form action="/upload_file" enctype="multipart/form-data" method="post">
          <input name="file" type="file" accept="video/*" required />
          <button type="submit">Upload</button>
        </form>
        <p>Setelah submit, server akan menyimpan file dan mengembalikan JSON berisi URL file.</p>
      </body>
    </html>
    '''
    return HTMLResponse(content=html, status_code=200)

In [ ]:
# Jalankan server uvicorn di dalam notebook dan buka tunnel publik via ngrok (opsional)
import nest_asyncio
from pyngrok import ngrok
import uvicorn
import asyncio

nest_asyncio.apply()
PORT = 8888

try:
    public_url = ngrok.connect(PORT)
    print('Public URL (ngrok):', public_url)
except Exception as e:
    print('ngrok connect gagal atau belum dikonfigurasi (authtoken).', e)

config = uvicorn.Config(app=app, host='0.0.0.0', port=PORT, log_level='info')
server = uvicorn.Server(config=config)

print('Menjalankan server uvicorn... (Ctrl+C untuk berhenti)')
async def run_server():
    await server.serve()

loop = asyncio.get_event_loop()
task = loop.create_task(run_server())
print('Server task started.')

t=2025-11-14T15:04:40+0700 lvl=eror msg="failed to reconnect session" obj=tunnels.session err="authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n"
t=2025-11-14T15:04:40+0700 lvl=eror msg="session closing" obj=tunnels.session err="authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n"
t=2025-11-14T15:04:40+0700 lvl=eror msg="terminating with error" obj=app err="authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018

ngrok connect gagal atau belum dikonfigurasi (authtoken). The ngrok process errored on start: authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n.
Menjalankan server uvicorn... (Ctrl+C untuk berhenti)
Server task started.


INFO:     Started server process [49552]
INFO:     Waiting for application startup.
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8888 (Press CTRL+C to quit)
INFO:     Uvicorn running on http://0.0.0.0:8888 (Press CTRL+C to quit)
C:\Users\Daffa\AppData\Local\Temp\ipykernel_49552\1245441747.py:32: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  safe_name = f"{datetime.utcnow().strftime('%Y%m%d%H%M%S')}_{uuid.uuid4().hex}{ext}"
C:\Users\Daffa\AppData\Local\Temp\ipykernel_49552\1245441747.py:32: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  safe_name = f"{date

INFO:     127.0.0.1:64481 - "POST /upload_file HTTP/1.1" 200 OK
INFO:     127.0.0.1:49232 - "POST /upload_file HTTP/1.1" 200 OK
INFO:     127.0.0.1:49232 - "POST /upload_file HTTP/1.1" 200 OK
INFO:     127.0.0.1:53788 - "POST /upload_file HTTP/1.1" 200 OK
INFO:     127.0.0.1:53788 - "POST /upload_file HTTP/1.1" 200 OK
INFO:     127.0.0.1:53788 - "OPTIONS /upload HTTP/1.1" 200 OK
INFO:     127.0.0.1:53788 - "OPTIONS /upload HTTP/1.1" 200 OK
INFO:     127.0.0.1:53788 - "POST /upload HTTP/1.1" 200 OK
INFO:     127.0.0.1:53788 - "POST /upload HTTP/1.1" 200 OK


C:\Users\Daffa\AppData\Local\Temp\ipykernel_49552\1245441747.py:63: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  fname = f"payload_{datetime.utcnow().strftime('%Y%m%d%H%M%S')}_{uuid.uuid4().hex}.json"
C:\Users\Daffa\AppData\Local\Temp\ipykernel_49552\1245441747.py:74: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  'saved_at': datetime.utcnow().isoformat()


Catatan:
- Akses lokal: http://127.0.0.1:8888
- Untuk menguji upload melalui browser buka: http://127.0.0.1:8888/upload_form
- Jika ngrok berhasil, public URL akan dicetak dan file yang diupload tersedia di <public_url>/uploads/<filename>
- Payload yang diterima melalui `/upload` disimpan di folder `received_payloads/`.